# Requirements

In [ ]:
!pip install spacy

In [1]:
!mkdir model #model folder

# Format corpus annotation to Spacy format

In [1]:
import os

"""
Read a corpus folder and zip ann
and corpus
-----Return---
zip(ann_txt)

"""

def zip_ann_txt(directory):
    txt_files = []
    ann_files = []
    entries = os.listdir(directory)
    #get sorted ann file
    for entry in sorted(entries):
        if entry.endswith(".txt"):
            txt_file = os.path.join(directory, entry)
            txt_files.append(txt_file)
        elif entry.endswith(".ann"):
            ann_file = os.path.join(directory, entry)
            ann_files.append(ann_file)
    return list(zip(ann_files,txt_files))
    
            
            
            
    
   
    
"""
Read MEDLINE corpus ann_file and his text file, map data to 
spacy ner training data format
"""
    
def format_brat_ann_txt_2_spacy_traningdata_line(ann_file, txt_file):
    i = 0
    training_line = ()
    dict_entities = {}
    entities_value = []
    empty_data = []
    #open annotation file
    with open(ann_file,'r') as ann_file_content:
        try:
            #split and get all line in the ann file
            ann = ann_file_content.read().strip().split("\n");
            """
            if ''in ann:
                print('yes')
                empty_data.append(os.path.splitext(ann_file)[0].split('/')[2])
                os.remove(ann_file)
            """
            # seek ann_file and get the useful line
            for i in range(0, len(ann) , 2):
                #print(ann[i].strip().split("\t"), 'with tab') 
                #split useful line with tab and format as tuple
                data = ann[i].strip().split("\t")[1]
                #print(data , 'with tab')
                data_tuple = tuple(data.split())
                #print(tuple(data.split()))
                #format tuple to spacy format
                renverse_tuple = (int(data_tuple[1]),int(data_tuple[2]),data_tuple[0])                
                #print(renverse_tuple)
                entities_value.append(renverse_tuple)
            #print(entities_value)
            dict_entities["entities"] = entities_value 
            if dict_entities.get("entities") == []:
                print('eeee')
            
        except IndexError:           
            pass

   # print(empty_data)
    #open txt_file file
    with open(txt_file,'r') as txt_file_content:
        try:
            #read a text
            text = txt_file_content.read().strip().split("\n")[0];
            """
            for elt in empty_data:
                if os.path.splitext(txt_file)[0].split('/')[2] == elt :
                    print('oui',txt_file)
                    os.remove(txt_file)
            """
            #create spacy traing data line
        except IndexError:
            pass
    training_line = (text,dict_entities)
    #print(text)
    #print(training_line)
    return training_line


"""
create a training data
"""
def create_training_data(corpus_dir):
    train_data = []
    zip_list = zip_ann_txt(corpus_dir)
    for ann, txt in zip_list: 
        print(ann)
        training_line = format_brat_ann_txt_2_spacy_traningdata_line(ann,txt)
        train_data.append(training_line)
    return train_data
    
        
        
    

# Test of data fromating

In [2]:
#read_corpus('corpus/MEDLINE')
#format_brat_ann_txt_2_spacy_traningdata_line('corpus/MEDLINE/9275861.ann','corpus/MEDLINE/9275861.txt')
print("--------------------")
TRAIN_DATA = create_training_data('corpus/MEDLINE')



--------------------
corpus/MEDLINE/10051979.ann
corpus/MEDLINE/1006003.ann
corpus/MEDLINE/10216975.ann
corpus/MEDLINE/1022186.ann
corpus/MEDLINE/10223112.ann
corpus/MEDLINE/1022429.ann
corpus/MEDLINE/102294.ann
corpus/MEDLINE/10230418.ann
corpus/MEDLINE/1025479.ann
corpus/MEDLINE/10361460.ann
corpus/MEDLINE/10427855.ann
corpus/MEDLINE/10442066.ann
corpus/MEDLINE/1052136.ann
corpus/MEDLINE/10530346.ann
corpus/MEDLINE/10599272.ann
corpus/MEDLINE/10686969.ann
corpus/MEDLINE/10758712.ann
corpus/MEDLINE/1076152.ann
corpus/MEDLINE/10765357.ann
corpus/MEDLINE/1079589.ann
corpus/MEDLINE/1087947.ann
corpus/MEDLINE/10928794.ann
corpus/MEDLINE/10992119.ann
corpus/MEDLINE/11008501.ann
corpus/MEDLINE/11155728.ann
corpus/MEDLINE/111870.ann
corpus/MEDLINE/11217573.ann
corpus/MEDLINE/11221361.ann
corpus/MEDLINE/11261311.ann
corpus/MEDLINE/11275616.ann
corpus/MEDLINE/11283453.ann
corpus/MEDLINE/11351220.ann
corpus/MEDLINE/11436597.ann
corpus/MEDLINE/11484458.ann
corpus/MEDLINE/11503488.ann
corpus/MEDL

# Training Ner model based on train_data

In [3]:
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


# training data
TRAIN_DATA = create_training_data('corpus/MEDLINE')

@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    output_dir=("Optional output directory", "option", "o", 'model'),
    n_iter=("Number of training iterations", "option", "n", 10),
)
def main(model=None, output_dir=None, n_iter=10):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
        for _, annotations in TRAIN_DATA:
            print(_,annotations)
            for ent in annotations.get("entities"):
                ner.add_label(ent[2])
    

    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)
    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
  

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
            print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])



corpus/MEDLINE/10051979.ann
corpus/MEDLINE/1006003.ann
corpus/MEDLINE/10216975.ann
corpus/MEDLINE/1022186.ann
corpus/MEDLINE/10223112.ann
corpus/MEDLINE/1022429.ann
corpus/MEDLINE/102294.ann
corpus/MEDLINE/10230418.ann
corpus/MEDLINE/1025479.ann
corpus/MEDLINE/10361460.ann
corpus/MEDLINE/10427855.ann
corpus/MEDLINE/10442066.ann
corpus/MEDLINE/1052136.ann
corpus/MEDLINE/10530346.ann
corpus/MEDLINE/10599272.ann
corpus/MEDLINE/10686969.ann
corpus/MEDLINE/10758712.ann
corpus/MEDLINE/1076152.ann
corpus/MEDLINE/10765357.ann
corpus/MEDLINE/1079589.ann
corpus/MEDLINE/1087947.ann
corpus/MEDLINE/10928794.ann
corpus/MEDLINE/10992119.ann
corpus/MEDLINE/11008501.ann
corpus/MEDLINE/11155728.ann
corpus/MEDLINE/111870.ann
corpus/MEDLINE/11217573.ann
corpus/MEDLINE/11221361.ann
corpus/MEDLINE/11261311.ann
corpus/MEDLINE/11275616.ann
corpus/MEDLINE/11283453.ann
corpus/MEDLINE/11351220.ann
corpus/MEDLINE/11436597.ann
corpus/MEDLINE/11484458.ann
corpus/MEDLINE/11503488.ann
corpus/MEDLINE/11565223.ann
corp

# Training

In [6]:
main('fr_core_news_sm','model/med_model_org',1000)

Loaded model 'fr_core_news_sm'
Diverticules et kystes arachnoïdiens rachidiens intraduraux . 6 observations . {'entities': [(0, 12, 'DISO'), (16, 36, 'DISO'), (37, 47, 'ANAT'), (48, 59, 'ANAT'), (64, 76, 'PROC')]}
Monitorage des efforts expulsifs pendant l' accouchement par l' électromyogramme des muscles intercostaux : étude pilote . {'entities': [(0, 10, 'PROC'), (15, 32, 'DISO'), (44, 56, 'PHYS'), (64, 80, 'PROC'), (85, 92, 'ANAT'), (108, 120, 'PROC')]}
Modalités des traitements locaux et généraux , médicamenteux ou non , dans la prise en charge du nouveau-né suspect d' être infecté / contaminé par HSV1 ou HSV2 . {'entities': [(14, 25, 'PROC'), (47, 60, 'CHEM'), (97, 107, 'LIVB'), (148, 152, 'DISO')]}
Résultats préliminaires de la technique d' excision du ptérygion avec autogreffe conjonctivale . {'entities': [(0, 9, 'PHEN'), (43, 64, 'PROC'), (70, 80, 'CHEM'), (81, 94, 'ANAT')]}
Le traitement chirurgical du strabisme divergent . {'entities': [(3, 13, 'PROC'), (29, 48, 'DISO')]}
Hyp

Losses {'ner': 8611.604720234871}
Losses {'ner': 7995.777644813061}
Losses {'ner': 7944.320419490337}
Losses {'ner': 7887.449908792973}
Losses {'ner': 7829.6060455441475}
Losses {'ner': 7736.64615303278}
Losses {'ner': 7760.102807044983}
Losses {'ner': 7697.016329705715}
Losses {'ner': 7717.709214031696}
Losses {'ner': 7726.578159734607}
Losses {'ner': 7698.6477419734}
Losses {'ner': 7588.633026093245}
Losses {'ner': 7581.764989614487}
Losses {'ner': 7641.206067383289}
Losses {'ner': 7660.571465939283}
Losses {'ner': 7694.453846827149}
Losses {'ner': 7597.242574870586}
Losses {'ner': 7651.76192510128}
Losses {'ner': 7643.206735551357}
Losses {'ner': 7687.744455516338}
Losses {'ner': 7624.514260053635}
Losses {'ner': 7608.783529996872}
Losses {'ner': 7622.004391789436}
Losses {'ner': 7591.742198288441}
Losses {'ner': 7589.522980809212}
Losses {'ner': 7577.515462934971}
Losses {'ner': 7584.323945611715}
Losses {'ner': 7549.462160170078}
Losses {'ner': 7561.275765776634}
Losses {'ner': 75

Losses {'ner': 7110.741226553917}
Losses {'ner': 7166.724843144417}
Losses {'ner': 7110.84733825922}
Losses {'ner': 7222.9855652451515}
Losses {'ner': 7118.348031878471}
Losses {'ner': 7154.864626586437}
Losses {'ner': 7167.858768999577}
Losses {'ner': 7084.696997977793}
Losses {'ner': 7247.50797227025}
Losses {'ner': 7102.463138639927}
Losses {'ner': 7159.39551932551}
Losses {'ner': 7107.745249807835}
Losses {'ner': 7134.880174666643}
Losses {'ner': 7153.8317076563835}
Losses {'ner': 7225.125811561942}
Losses {'ner': 7239.548661530018}
Losses {'ner': 7159.190977960825}
Losses {'ner': 7204.872921854258}
Losses {'ner': 7179.4650230407715}
Losses {'ner': 7200.656254573725}
Losses {'ner': 7162.678103327751}
Losses {'ner': 7211.328250654042}
Losses {'ner': 7170.5448122397065}
Losses {'ner': 7100.067651987076}
Losses {'ner': 7174.851140350103}
Losses {'ner': 7197.595059722662}
Losses {'ner': 7112.037760026753}
Losses {'ner': 7219.043442726135}
Losses {'ner': 7138.5239624381065}
Losses {'ner

Losses {'ner': 7107.186068713665}
Losses {'ner': 7170.125616788864}
Losses {'ner': 7044.665891230106}
Losses {'ner': 7066.839635595679}
Losses {'ner': 7031.140868246555}
Losses {'ner': 7090.537328332663}
Losses {'ner': 7115.111478418112}
Losses {'ner': 7088.698956251144}
Losses {'ner': 7128.594174019992}
Losses {'ner': 7122.874373696744}
Losses {'ner': 7087.5481832027435}
Losses {'ner': 7003.673495784402}
Losses {'ner': 7038.148371040821}
Losses {'ner': 7058.225075721741}
Losses {'ner': 7158.6051179766655}
Losses {'ner': 7103.896730393171}
Losses {'ner': 7188.121036887169}
Losses {'ner': 7085.373175948858}
Losses {'ner': 7137.618665099144}
Losses {'ner': 7090.2590975910425}
Losses {'ner': 7086.598985671997}
Losses {'ner': 7134.572032511234}
Losses {'ner': 7063.4970297608525}
Losses {'ner': 7069.992429152131}
Losses {'ner': 7132.048381836154}
Losses {'ner': 7070.830540060997}
Losses {'ner': 7137.43922868371}
Losses {'ner': 7082.102731496096}
Losses {'ner': 7112.036652222276}
Losses {'ne

Losses {'ner': 7067.1112544722855}
Losses {'ner': 7098.864970996976}
Losses {'ner': 7047.961788415909}
Losses {'ner': 6992.0500484108925}
Losses {'ner': 6977.371333032846}
Losses {'ner': 7072.432890146971}
Losses {'ner': 7060.427766621113}
Losses {'ner': 7094.904122538865}
Losses {'ner': 7115.575908601284}
Losses {'ner': 7154.974139511585}
Losses {'ner': 6992.911914169788}
Losses {'ner': 7104.36188700702}
Losses {'ner': 7084.284763097763}
Losses {'ner': 6974.759154729545}
Losses {'ner': 7050.216032519937}
Losses {'ner': 7089.192198932171}
Losses {'ner': 6991.994712769985}
Losses {'ner': 7079.552773296833}
Losses {'ner': 7040.807170689106}
Losses {'ner': 7100.067623972893}
Losses {'ner': 6891.490825727582}
Losses {'ner': 6988.258484192193}
Losses {'ner': 7036.357603490353}
Losses {'ner': 7058.605523914099}
Losses {'ner': 7131.1586537361145}
Losses {'ner': 6979.225956171751}
Losses {'ner': 7011.134084761143}
Losses {'ner': 6968.136166211218}
Losses {'ner': 6970.361954778433}
Losses {'ner

...

Entities [('vision', 'PHYS'), ('couleurs', 'PHEN'), ('potentiels évoqués', 'PHYS'), ('visuels', 'PHYS'), ('sclérose', 'DISO')]
Tokens [('Tests', '', 2), ('de', '', 2), ('la', '', 2), ('vision', 'PHYS', 3), ('des', '', 2), ('couleurs', 'PHEN', 3), ('et', '', 2), ('potentiels', 'PHYS', 3), ('évoqués', 'PHYS', 1), ('visuels', 'PHYS', 3), ('dans', '', 2), ('la', '', 2), ('sclérose', 'DISO', 3), ('en', '', 2), ('plaques', '', 2), ('.', '', 2)]
Entities [('hisien', 'ANAT'), ('troubles', 'DISO'), ('conduction', 'PHEN'), ('cardiaque', 'ANAT')]
Tokens [('Intérêt', '', 2), ('de', '', 2), ('l', '', 2), ("'", '', 2), ('enregistrement', '', 2), ('du', '', 2), ('potentiel', '', 2), ('hisien', 'ANAT', 3), ('avec', '', 2), ('épreuves', '', 2), ('pharmacodynamiques', '', 2), ('dans', '', 2), ('les', '', 2), ('troubles', 'DISO', 3), ('de', '', 2), ('la', '', 2), ('conduction', 'PHEN', 3), ('cardiaque', 'ANAT', 3)]
Entities [('SIDA', 'DISO')]
Tokens [('Le', '', 2), ('SIDA', 'DISO', 3), (':', '', 2), ('as

Entities [('corticosurrénales', 'ANAT'), ('hypertension', 'DISO'), ('artériosclérose', 'DISO'), ('humaine', 'LIVB')]
Tokens [('Le', '', 2), ('róle', '', 2), ('des', '', 2), ('corticosurrénales', 'ANAT', 3), ('dans', '', 2), ('l', '', 2), ("'", '', 2), ('hypertension', 'DISO', 3), ('et', '', 2), ('l', '', 2), ("'", '', 2), ('artériosclérose', 'DISO', 3), ('humaine', 'LIVB', 3), ('.', '', 2)]
Entities [('résultats', 'DISO')]
Tokens [('Premiers', '', 2), ('résultats', 'DISO', 3), ('d', '', 2), ("'", '', 2), ('une', '', 2), ('consultation', '', 2), ('de', '', 2), ('rhumatologie', '', 2), ('dans', '', 2), ('un', '', 2), ('grand', '', 2), ('centre', '', 2), ('administratif', '', 2), ('.', '', 2)]
Entities [('Lipome', 'DISO'), ('rétro-péritonéal', 'ANAT')]
Tokens [('Lipome', 'DISO', 3), ('rétro-péritonéal', 'ANAT', 3), ('géant', '', 2), ('.', '', 2)]
Entities [('Migration', 'PHYS'), ('douleur', 'DISO'), ('recherche épidémiologique', 'PROC')]
Tokens [('Migration', 'PHYS', 3), ('et', '', 2), ('

Entities [('traitement', 'PROC'), ('hémorragie', 'DISO')]
Tokens [('Apport', '', 2), ('de', '', 2), ('la', '', 2), ('neurochirurgie', '', 2), ('au', '', 2), ('traitement', 'PROC', 3), ('de', '', 2), ('l', '', 2), ("'", '', 2), ('hémorragie', 'DISO', 3), ('intracérébrale', '', 2)]
Entities [('Interprétation', 'PROC'), ('résultats', 'PHEN'), ('gazeuse', 'CHEM')]
Tokens [('Interprétation', 'PROC', 3), ('des', '', 2), ('résultats', 'PHEN', 3), ('de', '', 2), ('la', '', 2), ('méniscographie', '', 2), ('gazeuse', 'CHEM', 3), ('.', '', 2)]
Entities [('colorations', 'PROC'), ('aiguës', 'DISO')]
Tokens [('Apport', '', 2), ('des', '', 2), ('doubles', '', 2), ('colorations', 'PROC', 3), ('dans', '', 2), ('la', '', 2), ('cytochimie', '', 2), ('des', '', 2), ('leucoses', '', 2), ('aiguës', 'DISO', 3), ('.', '', 2)]
Entities [('Administration', 'PROC'), ('solution sucrée', 'CHEM'), ('maternel', 'PHYS')]
Tokens [('1', '', 2), ('/', '', 2), ('6', '', 2), ('Administration', 'PROC', 3), ('d', '', 2), ("

Entities [('Etude', 'PROC'), ('marqueurs', 'CHEM'), ('HBV', 'DISO'), ('lymphocytaires', 'ANAT'), ('polytoxicomane', 'LIVB'), ('asymptomatique', 'DISO')]
Tokens [('Etude', 'PROC', 3), ('des', '', 2), ('marqueurs', 'CHEM', 3), ('de', '', 2), ('l', '', 2), ("'", '', 2), ('HBV', 'DISO', 3), ('et', '', 2), ('des', '', 2), ('populations', '', 2), ('lymphocytaires', 'ANAT', 3), ('chez', '', 2), ('le', '', 2), ('polytoxicomane', 'LIVB', 3), ('asymptomatique', 'DISO', 3), ('.', '', 2)]
Entities [('Pharmacodynamie', 'PHEN'), ('anesthésiques', 'CHEM'), ('chirurgien-dentiste', 'LIVB')]
Tokens [('Pharmacodynamie', 'PHEN', 3), ('des', '', 2), ('anesthésiques', 'CHEM', 3), ('locaux', '', 2), ('du', '', 2), ('chirurgien', 'LIVB', 3), ('-', 'LIVB', 1), ('dentiste', 'LIVB', 1), ('.', '', 2)]
Entities [('transfusionnel', 'PROC')]
Tokens [('Évolutions', '', 2), ('technologiques', '', 2), ('en', '', 2), ('qualification', '', 2), ('biologique', '', 2), ('du', '', 2), ('don', '', 2), ('et', '', 2), ('leur', 

Entities [('surdités', 'DISO'), ('transmission', 'DISO'), ('adulte', 'LIVB')]
Tokens [('Les', '', 2), ('surdités', 'DISO', 3), ('de', '', 2), ('transmission', 'DISO', 3), ('de', '', 2), ('l', '', 2), ("'", '', 2), ('adulte', 'LIVB', 3), ('.', '', 2), ('Eléments', '', 2), ('diagnostiques', '', 2), ('et', '', 2), ('principes', '', 2), ('thérapeutiques', '', 2), ('.', '', 2)]
Entities [('protéines', 'CHEM'), ('sériques', 'ANAT'), ('liquide', 'ANAT'), ('synovial', 'ANAT'), ('polyarthrite', 'DISO'), ('rhumatoide', 'CHEM')]
Tokens [('Modifications', '', 2), ('des', '', 2), ('protéines', 'CHEM', 3), ('sériques', 'ANAT', 3), ('et', '', 2), ('du', '', 2), ('liquide', 'ANAT', 3), ('synovial', 'ANAT', 3), ('au', '', 2), ('cours', '', 2), ('de', '', 2), ('la', '', 2), ('polyarthrite', 'DISO', 3), ('rhumatoide', 'CHEM', 3), ('.', '', 2)]
Entities [('soins', 'PROC')]
Tokens [('Vers', '', 2), ('un', '', 2), ('contrôle', '', 2), ('de', '', 2), ('la', '', 2), ('qualité', '', 2), ('des', '', 2), ('soins